# CARTOLA FC - 2020

Cartola FC é um jogo eletrônico de futebol no estilo fantasy game, um jogo fictício no qual as pessoas montam seus times com jogadores de futebol da vida real, lançado no ano de 2004. Foi criado e é mantido pelo site Globo.com e promovido pelo canal de TV por assinatura SporTV e também pela Globo.

<hr>

## Definição do Problema

### Previsão de Escalação

Realizar previsão da pontuação dos jogadores do Cartola FC.

<hr>

## Conjunto de dados

In [4]:
# analise de dados
import pandas as pd
import numpy as np
import random as rnd

# Requests e Elementos HTML
import re                          # Expressão regulares
import requests                    # Acessar páginas da internet
from bs4 import BeautifulSoup      # Raspar elementos de páginas da internet

# visualização
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

# metricas 
from sklearn.metrics import classification_report

### Captura dos dados 

Os dados são baixados diretamente do diretório do do GitHub: https://github.com/henriquepgomide/caRtola/tree/master/data/2020

In [5]:

# URL com caminho do repositório
URL = 'https://github.com/henriquepgomide/caRtola/tree/master/data/2020'
html = requests.get(URL)
# Criar objeto BeautifulSoup para raspar urls 
soup = BeautifulSoup(html.text, 'lxml')

In [132]:
# Formatar HTML para extrair os dados
result = []
for tag in soup.find_all('a', href=True):
    result.append(tag)
result[70:90:2]

[<a class="f6 link-gray text-mono ml-2 d-none d-lg-inline" data-pjax="" href="/henriquepgomide/caRtola/commit/c0dc67a7bee213b75edb32a5e409e3e88fb51ece">c0dc67a</a>,
 <a class="link-gray-dark text-bold" data-pjax="true" href="/henriquepgomide/caRtola/commit/c0dc67a7bee213b75edb32a5e409e3e88fb51ece">add team features, new round, update aggregated statistics</a>,
 <a class="d-none js-permalink-shortcut" data-hotkey="y" href="/henriquepgomide/caRtola/tree/c0dc67a7bee213b75edb32a5e409e3e88fb51ece/data/2020">Permalink</a>,
 <a class="js-navigation-open link-gray-dark" href="/henriquepgomide/caRtola/tree/master/data/2020/team-features" id="015cc237790ac744a369fe0a639f3b4b-40cbb9d10b172966a10d0cc0beb858725ca89218" title="team-features">team-features</a>,
 <a class="js-navigation-open link-gray-dark" href="/henriquepgomide/caRtola/tree/master/data/2020/team-rankings" id="4c3fa43e2bc40689660a8b10cb303c0a-e9fa230a75ec759e35a03d712be5e14549625e2f" title="team-rankings">team-rankings</a>,
 <a class

In [133]:
regex =  '\.csv$'

dict_of_files = {}                                                          # Criar dicionário vazio
for tag in soup.find_all('a', attrs={'href': re.compile(regex)}):           # Encontrar tags de nosso interesse
    href_str = tag.get('href')                                              
    file_name = re.sub('/henriquepgomide/caRtola/blob/master/data/2020/',   # Substituir padrão por nada
                       '', 
                       href_str)
    
    file_url = re.sub('/henriquepgomide/caRtola/blob/master/data/2020/',    # Substituir padrão por links para arquivos raw
                    'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/', 
                    href_str)

    dict_of_files[file_name] = file_url

In [134]:
dict(list(dict_of_files.items())) # Ignore este código horrível

{'2020-medias-jogadores.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/2020-medias-jogadores.csv',
 '2020_partidas.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/2020_partidas.csv',
 'rodada-1.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-1.csv',
 'rodada-10.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-10.csv',
 'rodada-11.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-11.csv',
 'rodada-2.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-2.csv',
 'rodada-3.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-3.csv',
 'rodada-4.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-4.csv',
 'rodada-5.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rod

In [151]:
# Ler os dataframes das rodadas
list_of_dataframes = []
for key, item in list(dict_of_files.items())[2:14]:
    df = pd.read_csv(item)
    df['rodada'] = key
    list_of_dataframes.append(df)

In [152]:
# Combinar lista de DataFrames criados
df_jogadores = pd.concat(list_of_dataframes)
df_jogadores.shape


(8573, 35)

In [153]:
# Ler os dataframes das partidas 
for key, item in list(dict_of_files.items())[1:2]:
    df_jogos = pd.read_csv(item)


In [154]:
df_jogos.shape

(119, 6)

In [156]:
df_jogadores.tail(3)

,Unnamed: 0,atletas.nome,atletas.slug,atletas.apelido,atletas.foto,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,atletas.clube.id.full.name,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,rodada,DP
709,710,Cayo Henrique Nascimento Ferreira,cayo-tenorio,Cayo Tenório,https://s.glbimg.com/es/sde/f/2020/08/17/41541...,107367,9,267,lat,Nulo,0,4.88,0,4.33,4,Vasco,1,7,NaN,17,2,5,5,NaN,NaN,NaN,1,NaN,3,NaN,NaN,NaN,NaN,rodada-9.csv,NaN
710,711,Caio Alan Tem Catem Gonçalves,caio,Caio,https://s.glbimg.com/es/sde/f/2018/03/01/87be2...,99460,9,293,gol,Nulo,0,1,0,0,0,Athlético-PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-9.csv,NaN
711,712,Luiz Henrique André Rosa da Silva,luiz-henrique,Luiz Henrique,https://s.glbimg.com/es/sde/f/2020/08/11/4a557...,109282,9,266,ata,Nulo,0.2,1.5,-0.48,1.01,7,Fluminense,5,1,NaN,14,NaN,4,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-9.csv,NaN


In [157]:
df_jogadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8573 entries, 0 to 711
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  8572 non-null   object
 1   atletas.nome                8571 non-null   object
 2   atletas.slug                8571 non-null   object
 3   atletas.apelido             8571 non-null   object
 4   atletas.foto                8504 non-null   object
 5   atletas.atleta_id           8571 non-null   object
 6   atletas.rodada_id           8571 non-null   object
 7   atletas.clube_id            8571 non-null   object
 8   atletas.posicao_id          8571 non-null   object
 9   atletas.status_id           8571 non-null   object
 10  atletas.pontos_num          8571 non-null   object
 11  atletas.preco_num           8571 non-null   object
 12  atletas.variacao_num        8571 non-null   object
 13  atletas.media_num           8571 non-null   objec

In [158]:
df_jogos.tail(2)

,date,home_team,away_team,home_score,away_score,round
117,2020-09-27,354,290,NaN,NaN,12
118,2020-09-27,373,263,NaN,NaN,12


In [159]:
df_jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        119 non-null    object 
 1   home_team   119 non-null    int64  
 2   away_team   119 non-null    int64  
 3   home_score  103 non-null    float64
 4   away_score  103 non-null    float64
 5   round       119 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 5.7+ KB


In [160]:
# Remover colunas atletas.foto e rodada
df_jogadores.drop(['atletas.foto', 'rodada', 'atletas.slug', 'atletas.nome', 'atletas.clube.id.full.name'], axis=1, inplace=True)

,Unnamed: 0,atletas.apelido,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,DP
0,1,Jorginho,77544,1,373,mei,Provável,0,6,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Vagner Mancini,39850,1,373,tec,Provável,0,2,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Moacir,60858,1,373,mei,Nulo,0,3,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Léo Cittadini,70986,1,293,mei,Provável,9.8,12.9,4.9,9.8,1,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Alex Muralha,79066,1,294,gol,Nulo,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,708,Mano Menezes,37281,9,265,tec,Provável,0,8.06,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,709,Matheus Alves,91712,9,290,gol,Nulo,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,710,Cayo Tenório,107367,9,267,lat,Nulo,0,4.88,0,4.33,4,1,7,NaN,17,2,5,5,NaN,NaN,NaN,1,NaN,3,NaN,NaN,NaN,NaN,NaN
710,711,Caio,99460,9,293,gol,Nulo,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
# remover linhas cuja coluna rodada_id esteja NaN
df_jogadores.dropna(subset=['atletas.rodada_id'], inplace=True)

In [114]:
 # preencher com 0 colunas NaN
 #df_jogadores.fillna(0, inplace=True)
 # Remover coluna Unnamed
 #df_jogadores = df_jogadores.loc[:, ~df_jogadores.columns.str.contains('^Unnamed')]

In [150]:
# Remover técnicos do dataset
#df_jogadores = df_jogadores.drop(df_jogadores.loc[df_jogadores['atletas.posicao_id'] == 'tec'].index)

In [149]:
df_jogadores.shape

(6468, 30)

In [ ]:
#df_jogadores.dropna(subset=['atletas.rodada_id'], inplace=True)

In [117]:
# Verificar dados NaN
df_jogadores.isna().sum()

atletas.apelido            1
atletas.atleta_id          1
atletas.rodada_id          1
atletas.clube_id           1
atletas.posicao_id         1
atletas.status_id          1
atletas.pontos_num         1
atletas.preco_num          1
atletas.variacao_num       1
atletas.media_num          1
atletas.jogos_num          1
FF                      4259
FS                      3377
G                       5647
PI                      2605
CA                      4787
FC                      3185
DS                      3198
FT                      6121
DD                      6243
GS                      6229
FD                      4783
GC                      6435
SG                      5516
A                       5801
I                       5547
CV                      6305
PP                      6425
DP                      6452
dtype: int64

In [64]:
df_jogos.shape

(119, 6)

In [65]:
# Verificar dados NaN no dataset das Partidas
df_jogos[df_jogos.isna().any(axis=1)]

,date,home_team,away_team,home_score,away_score,round
1,2020-09-30,263,265,NaN,NaN,12
2,2020-08-09,275,267,NaN,NaN,1
4,2020-09-30,264,373,NaN,NaN,12
9,2020-08-09,290,276,NaN,NaN,1
54,2020-08-29,282,293,NaN,NaN,6
55,2020-08-30,284,290,NaN,NaN,6
109,2020-09-28,266,294,NaN,NaN,12
110,2020-09-27,267,280,NaN,NaN,12
111,2020-09-27,275,262,NaN,NaN,12
112,2020-09-27,277,356,NaN,NaN,12


In [161]:
df_jogadores[(df_jogadores['atletas.clube_id'] == 262) & (df_jogadores['atletas.atleta_id'] == 38509)]

,Unnamed: 0,atletas.nome,atletas.slug,atletas.apelido,atletas.foto,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,atletas.clube.id.full.name,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,rodada,DP
11,12,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,1,262,gol,Provável,5.6,14.21,0.21,5.6,1,Flamengo,NaN,1,NaN,9,NaN,NaN,NaN,NaN,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv,NaN
502,503,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,11,262,gol,Contundido,0,10.72,0,2.22,5,Flamengo,NaN,2,NaN,19,NaN,NaN,NaN,NaN,6,6,NaN,NaN,1,NaN,NaN,1,NaN,rodada-11.csv,NaN
110,111,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,2,262,gol,Suspenso,-6.5,9.28,-4.93,-0.45,2,Flamengo,NaN,2,NaN,9,NaN,NaN,NaN,NaN,3,4,NaN,NaN,NaN,NaN,NaN,1,NaN,rodada-2.csv,NaN
386,387,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,3,262,gol,Provável,0,9.28,0,-0.45,2,Flamengo,NaN,2,NaN,9,NaN,NaN,NaN,NaN,3,4,NaN,NaN,NaN,NaN,NaN,1,NaN,rodada-3.csv,NaN
128,129,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,4,262,gol,Provável,-2.4,7.33,-1.95,-1.1,3,Flamengo,NaN,2,NaN,13,NaN,NaN,NaN,NaN,3,5,NaN,NaN,NaN,NaN,NaN,1,NaN,rodada-4.csv,NaN
28,29,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,5,262,gol,Provável,-2.1,6.58,-0.75,-1.35,4,Flamengo,NaN,2,NaN,14,NaN,NaN,NaN,NaN,3,6,NaN,NaN,NaN,NaN,NaN,1,NaN,rodada-5.csv,NaN
681,682,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,6,262,gol,Dúvida,16.5,10.72,4.14,2.22,5,Flamengo,NaN,2,NaN,19,NaN,NaN,NaN,NaN,6,6,NaN,NaN,1,NaN,NaN,1,NaN,rodada-6.csv,NaN
62,63,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,7,262,gol,Contundido,0,10.72,0,2.22,5,Flamengo,NaN,2,NaN,19,NaN,NaN,NaN,NaN,6,6,NaN,NaN,1,NaN,NaN,1,NaN,rodada-7.csv,NaN
85,86,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,8,262,gol,Contundido,0,10.72,0,2.22,5,Flamengo,NaN,2,NaN,19,NaN,NaN,NaN,NaN,6,6,NaN,NaN,1,NaN,NaN,1,NaN,rodada-8.csv,NaN
208,209,Diego Alves Carreira,diego-alves,Diego Alves,https://s.glbimg.com/es/sde/f/2020/07/21/b85ac...,38509,9,262,gol,Contundido,0,10.72,0,2.22,5,Flamengo,NaN,2,NaN,19,NaN,NaN,NaN,NaN,6,6,NaN,NaN,1,NaN,NaN,1,NaN,rodada-9.csv,NaN


In [66]:
# Remover linhas NaN pelo home score
df_jogos.dropna(subset=['home_score'], inplace=True)

In [67]:
# Renomear colunas que possuem a número da rodada para rodada_id em ambos dataset
df_jogos.rename(columns={'round':'rodada_id'}, inplace=True)
df_jogadores.rename(columns={'atletas.rodada_id':'rodada_id'}, inplace=True)

In [68]:
df_jogadores.tail(3)

,atletas.apelido,atletas.atleta_id,rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,DP
709,Cayo Tenório,107367,9,267,lat,Nulo,0,4.88,0,4.33,4,1,7,0,17,2,5,5,0,0,0,1,0,3,0,0,0,0,0
710,Caio,99460,9,293,gol,Nulo,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
711,Luiz Henrique,109282,9,266,ata,Nulo,0.2,1.5,-0.48,1.01,7,5,1,0,14,0,4,6,0,0,0,0,0,0,0,0,0,0,0


In [28]:
# Procura qual adversário por rodade e atleta
def getAdversario(clube_id, rodada_id):
    advFora = df_jogos.loc[(df_jogos['home_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['away_team']
    advCasa = df_jogos.loc[(df_jogos['away_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['home_team']
    if (advFora is not None) and (advFora.values > 0):
        return int(advFora.values[0])
    else :
         if (advCasa is not None) and (advCasa.values > 0):
               return int(advCasa.values[0])


In [29]:
# procura se o jogo foi em casa ou f
def getJogoForaEmCasa(clube_id, rodada_id):
    jogouEmCasa = df_jogos.loc[(df_jogos['home_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['home_team']
    jogouFora = df_jogos.loc[(df_jogos['away_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['away_team']
    if (jogouEmCasa is not None) and (jogouEmCasa.values > 0):
        return 0
    else :
         if (jogouFora is not None) and (jogouFora.values > 0):
               return 1


In [30]:
# Procura qual adversário por rodade e atleta
def getGolsFavorJogandoEmCasa(clube_id, rodada_id):
    golsFavorCasa = df_partidas.loc[(df_partidas['home_team'] == clube_id) & (df_partidas['rodada_id'] == rodada_id)]['home_score']
    if (golsFavorCasa is not None) and (golsFavorCasa.values.size > 0):
        return int(golsFavorCasa.values[0])

In [31]:
def gerarColunasGols(linha_df):
    jogoRealizadoEmCasa = df_jogos.loc[(df_jogos['home_team'] == linha_df['atletas.clube_id']) & (df_jogos['rodada_id'] == linha_df['rodada_id'])]
    jogoRealizadoFora = df_jogos.loc[(df_jogos['away_team'] == linha_df['atletas.clube_id']) & (df_jogos['rodada_id'] == linha_df['rodada_id'])]
    if not jogoRealizadoEmCasa.empty:
        linha_df['golFavor'] = jogoRealizadoEmCasa['home_score'].values[0]
        linha_df['golContra'] = jogoRealizadoEmCasa['away_score'].values[0]
    else:
        if not jogoRealizadoFora.empty:
            linha_df['golFavor'] = jogoRealizadoFora['away_score'].values[0]
            linha_df['golContra'] = jogoRealizadoFora['home_score'].values[0]
    return linha_df
        

In [32]:
# Gera novas colunas com os gols do time a favor ou contra para cada rodada
df_jogadores = df_jogadores.apply(lambda x: gerarColunasGols(x), axis=1)

In [33]:
# Adversário do time/jogador
df_jogadores['adversario'] = df_jogadores.apply(lambda x: getAdversario(x['atletas.clube_id'], x['rodada_id']), axis=1)

Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: Deprecat

In [34]:
# Jogo fora ou em casa
df_jogadores['jogoForaOuCasa'] = df_jogadores.apply(lambda x: getJogoForaEmCasa(x['atletas.clube_id'], x['rodada_id']), axis=1)

Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/home/linux/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: Deprecat

In [38]:
pd.set_option('display.max_columns', None)
df_jogadores.tail(30)

,A,CA,CV,DD,DP,DS,FC,FD,FF,FS,FT,G,GC,GS,I,PI,PP,SG,atletas.apelido,atletas.atleta_id,atletas.clube_id,atletas.jogos_num,atletas.media_num,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.status_id,atletas.variacao_num,golContra,golFavor,rodada_id,adversario,jogoForaOuCasa
658,0,1,0,0,0,5,3,0,1,6,0,0,0,0,0,10,0,0,Lucas Evangelista,84674,280,5,0.86,1.4,mei,3.06,Nulo,0.26,1.0,1.0,9,276.0,1.0
660,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,Kawan,106219,293,1,-2.6,0,mei,0.72,Nulo,0,1.0,1.0,9,263.0,0.0
662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Jeferson Douglas,109405,265,0,0,0,mei,1,Nulo,0,2.0,0.0,9,284.0,0.0
664,0,1,0,0,0,5,1,0,0,2,0,1,0,0,0,2,0,0,Luis Phelipe,104500,280,3,3.77,0,ata,4.64,Contundido,0,1.0,1.0,9,276.0,1.0
665,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rodrigues,106322,290,1,0,0,lat,1.48,Nulo,0,3.0,3.0,9,294.0,0.0
666,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fabrício,104134,284,1,0,0,ata,0.77,Nulo,0,0.0,2.0,9,265.0,1.0
671,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,João Fernando,104079,262,0,0,0,gol,1,Nulo,0,1.0,2.0,9,266.0,1.0
672,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lucas Ribeiro,103088,285,0,0,0,zag,4,Nulo,0,0.0,2.0,9,354.0,0.0
673,0,1,0,0,0,7,1,1,0,5,0,0,0,0,0,12,0,2,Neto Borges,106411,267,4,4.25,0,lat,5.32,Nulo,0,2.0,1.0,9,373.0,0.0
674,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Walisson Pequeno,101696,290,0,0,0,mei,2,Nulo,0,3.0,3.0,9,294.0,0.0


In [39]:
df_jogadores.isna().sum()

A                       0
CA                      0
CV                      0
DD                      0
DP                      0
DS                      0
FC                      0
FD                      0
FF                      0
FS                      0
FT                      0
G                       0
GC                      0
GS                      0
I                       0
PI                      0
PP                      0
SG                      0
atletas.apelido         0
atletas.atleta_id       0
atletas.clube_id        0
atletas.jogos_num       0
atletas.media_num       0
atletas.pontos_num      0
atletas.posicao_id      0
atletas.preco_num       0
atletas.status_id       0
atletas.variacao_num    0
golContra               0
golFavor                0
rodada_id               0
adversario              0
jogoForaOuCasa          0
dtype: int64

In [40]:
# remover linhas cuja coluna rodada_id esteja NaN
df_jogadores.dropna(subset=['adversario'], inplace=True)

In [491]:
df_jogadores.isna().sum()

A                       0
CA                      0
CV                      0
DD                      0
DP                      0
DS                      0
FC                      0
FD                      0
FF                      0
FS                      0
FT                      0
G                       0
GC                      0
GS                      0
I                       0
PI                      0
PP                      0
SG                      0
atletas.apelido         0
atletas.atleta_id       0
atletas.clube_id        0
atletas.jogos_num       0
atletas.media_num       0
atletas.pontos_num      0
atletas.posicao_id      0
atletas.preco_num       0
atletas.status_id       0
atletas.variacao_num    0
golContra               0
golFavor                0
rodada_id               0
adversario              0
jogoForaOuCasa          0
dtype: int64

In [492]:
df_jogadores.shape

(4757, 33)

In [41]:
df_jogadores.head(10)

,A,CA,CV,DD,DP,DS,FC,FD,FF,FS,FT,G,GC,GS,I,PI,PP,SG,atletas.apelido,atletas.atleta_id,atletas.clube_id,atletas.jogos_num,atletas.media_num,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.status_id,atletas.variacao_num,golContra,golFavor,rodada_id,adversario,jogoForaOuCasa
6,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,Sassá,79578,294,1,-2.6,-2.6,ata,2.46,Nulo,-3.54,1.0,0.0,1,285.0,0.0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Santos,69012,293,0,0,0,gol,10,Provável,0,0.0,2.0,1,356.0,1.0
11,0,0,0,2,0,0,0,0,0,1,0,0,0,1,0,9,0,0,Diego Alves,38509,262,1,5.6,5.6,gol,14.21,Provável,0.21,1.0,0.0,1,282.0,0.0
13,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,0,0,Rodrigo Caio,73800,262,1,0.7,0.7,zag,9.2,Provável,-2.8,1.0,0.0,1,282.0,0.0
15,0,0,0,0,0,2,2,0,0,1,0,0,1,0,0,12,0,0,Filipe Luís,38750,262,1,-4.7,-4.7,lat,3.84,Provável,-6.16,1.0,0.0,1,282.0,0.0
16,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,Miguel,105612,266,1,0.3,0.3,mei,2.39,Nulo,-0.61,1.0,0.0,1,284.0,1.0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cazares,81682,282,0,0,0,mei,12,Nulo,0,0.0,1.0,1,262.0,1.0
18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Roberto,104086,285,0,0,0,zag,2,Nulo,0,0.0,1.0,1,294.0,1.0
19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Diego,38909,262,0,0,0,mei,8,Nulo,0,1.0,0.0,1,282.0,0.0
21,0,0,0,0,0,5,0,0,0,2,0,0,0,0,0,5,0,1,Igor Rabello,89493,282,1,10.5,10.5,zag,13.41,Provável,5.41,0.0,1.0,1,262.0,1.0


In [95]:
df_jogadores[(df_jogadores['atletas.clube_id'] == 262) & (df_jogadores['atletas.atleta_id'] == 38509)]

,atletas.apelido,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,DP
11,Diego Alves,38509,1,262,gol,Provável,5.6,14.21,0.21,5.6,1,0,1,0,9,0,0,0,0,2,1,0,0,0,0,0,0,0,0
502,Diego Alves,38509,11,262,gol,Contundido,0,10.72,0,2.22,5,0,2,0,19,0,0,0,0,6,6,0,0,1,0,0,1,0,0
110,Diego Alves,38509,2,262,gol,Suspenso,-6.5,9.28,-4.93,-0.45,2,0,2,0,9,0,0,0,0,3,4,0,0,0,0,0,1,0,0
128,Diego Alves,38509,4,262,gol,Provável,-2.4,7.33,-1.95,-1.1,3,0,2,0,13,0,0,0,0,3,5,0,0,0,0,0,1,0,0
28,Diego Alves,38509,5,262,gol,Provável,-2.1,6.58,-0.75,-1.35,4,0,2,0,14,0,0,0,0,3,6,0,0,0,0,0,1,0,0
62,Diego Alves,38509,7,262,gol,Contundido,0,10.72,0,2.22,5,0,2,0,19,0,0,0,0,6,6,0,0,1,0,0,1,0,0
85,Diego Alves,38509,8,262,gol,Contundido,0,10.72,0,2.22,5,0,2,0,19,0,0,0,0,6,6,0,0,1,0,0,1,0,0
208,Diego Alves,38509,9,262,gol,Contundido,0,10.72,0,2.22,5,0,2,0,19,0,0,0,0,6,6,0,0,1,0,0,1,0,0


<hr>

### Análise Descritiva
A análise de dados descritiva apresenta a separação e os tipos das características bem como quais colunas possuem valores vazios. Além de apresentar uma visão resumida da distribuição do dataset.

#### Features categóricas x numéricas
- categóricas:
    - nominal: ``TARGET``

- numéricas: 
    - continua: ``LAT``, ``LONG``
    - discreta: ``Movimentação``, ``Original_473``, ``Original_269``, ``Zero``, ``Maçã-Verde``, ``Tangerina``, ``Citrus``, ``Açaí-Guaraná``, ``Pêssego``,``row``
 



#### Tipos das características

- Float: 2 
- Int64: 10
- Objetos: 3 (Strings)


#### Dados faltantas / nulos
- Não existem dados faltantes no dataset

In [5]:
#Informações gerais
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17016 entries, 0 to 17015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Tempo         17016 non-null  object 
 1   Estação       17016 non-null  object 
 2   LAT           17016 non-null  float64
 3   LONG          17016 non-null  float64
 4   Movimentação  17016 non-null  int64  
 5   Original_473  17016 non-null  int64  
 6   Original_269  17016 non-null  int64  
 7   Zero          17016 non-null  int64  
 8   Maçã-Verde    17016 non-null  int64  
 9   Tangerina     17016 non-null  int64  
 10  Citrus        17016 non-null  int64  
 11  Açaí-Guaraná  17016 non-null  int64  
 12  Pêssego       17016 non-null  int64  
 13  TARGET        17016 non-null  object 
dtypes: float64(2), int64(9), object(3)
memory usage: 1.8+ MB


<hr>

#### Distribuição numérica
- ``Original_473`` tem apróximadamente uma média 46 un.
- ``Original_473``,``Original_269``,``Zero`` já tiveram o mínimo de 0 un.

In [6]:
#Describe das variáveis numéricias
df.describe()

,LAT,LONG,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego
count,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000
mean,-23.572574,-46.652468,67674.478843,46.827045,36.036495,35.640691,24.339739,23.885696,24.428185,23.863540,23.904502
std,0.026361,0.033325,43791.284669,25.539867,18.787358,19.124799,12.089773,12.345912,12.072112,12.347261,12.308680
min,-23.626300,-46.725200,13592.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,-23.589500,-46.682300,33312.500000,25.000000,20.000000,19.000000,14.000000,13.000000,14.000000,13.000000,13.000000
50%,-23.566400,-46.648200,64194.500000,47.000000,36.000000,36.000000,24.000000,24.000000,25.000000,24.000000,24.000000
75%,-23.550000,-46.634600,89933.500000,68.000000,52.000000,52.000000,35.000000,35.000000,35.000000,35.000000,35.000000
max,-23.536500,-46.576500,195294.000000,86.000000,65.000000,65.000000,43.000000,43.000000,43.000000,43.000000,43.000000


#### Distribuição categórica
- A ``Estação`` Ana Rosa é a que mais possui registros
- No dia 24/08/2018 foi registrado a maior quantidade de registros (``tempo``)
- O maior quantidade ``target`` é o Normal 

In [7]:
#Describe das variáveis categóricas
df.describe(include=['O'])

,Tempo,Estação,TARGET
count,17016,17016,17016
unique,720,25,2
top,2018-8-24,Ana Rosa,NORMAL
freq,24,720,11518


<hr>


### Análise por características cruzadas

#### Features categóricas


In [8]:
#Total de registros por target
df['TARGET'].value_counts()

NORMAL         11518
REABASTECER     5498
Name: TARGET, dtype: int64

<hr>

### Análise por visualização de dados
Visualização para melhor interpretação e identificação de possíveis outliers.

<hr>

## Correção e Limpeza dos dados

### Desições tomadas após análise

- Remover as colunas ``row``, ``movimentacao``
- Adicionar mais amostras ao ``TARGET`` REABASTECER
- Visualizar matriz de correlação após correções.

### Pre-processando o dataset antes do treinamento

In [192]:
df = pd.read_csv('dataset.csv')
df_teste = pd.read_csv('to_be_scored.csv')

In [193]:
#total geral de unidades por linha
df['total_un'] = df.apply(lambda x: (x['Original_473'] + x['Original_269'] + x['Zero'] + x['Maçã-Verde'] + x['Tangerina'] + x['Citrus'] + x['Açaí-Guaraná'] + x['Pêssego'] ), axis=1)
df.loc[df['Tempo'] == '2018-2-30', 'Tempo'] = '2018-2-28'
df.loc[df['Tempo'] == '2018-2-29', 'Tempo'] = '2018-2-28'
df.loc[df['Tempo'] == '2019-2-30', 'Tempo'] = '2019-2-28'
df.loc[df['Tempo'] == '2019-2-29', 'Tempo'] = '2019-2-28'
df['Tempo'] = pd.to_datetime(df['Tempo'])


In [194]:
#Ajuste das labels
df = df.sort_values(['Estação','Tempo']).reset_index(drop=True)
for row in range(1, len(df)):
    if (row < 17014) and (df.loc[row, 'TARGET'] == 'NORMAL') and (df.loc[row+1, 'total_un'] > df.loc[row, 'total_un']):
        df.loc[row, 'TARGET'] = 'REABASTECER'


In [195]:
df = df.drop(['Tempo', 'Estação', 'Movimentação'],  axis=1)
df

,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET,total_un
0,-23.5813,-46.6383,86,65,65,43,43,43,43,43,NORMAL,431
1,-23.5813,-46.6383,75,60,55,41,37,43,37,37,NORMAL,385
2,-23.5813,-46.6383,68,54,45,38,31,40,31,31,NORMAL,338
3,-23.5813,-46.6383,57,45,37,37,25,35,25,29,NORMAL,290
4,-23.5813,-46.6383,45,42,30,36,23,32,20,24,NORMAL,252
...,...,...,...,...,...,...,...,...,...,...,...,...
17011,-23.5895,-46.6346,34,25,65,7,38,9,1,17,REABASTECER,196
17012,-23.5895,-46.6346,26,15,61,6,34,5,43,11,REABASTECER,201
17013,-23.5895,-46.6346,13,8,53,43,30,43,41,8,REABASTECER,239
17014,-23.5895,-46.6346,5,65,48,42,24,39,36,3,NORMAL,262


### Ajuste do DataFrame de Teste

In [197]:
#total geral de unidades por linha
df_teste['total_un'] = df_teste.apply(lambda x: (x['Original_473'] + x['Original_269'] + x['Zero'] + x['Maçã-Verde'] + x['Tangerina'] + x['Citrus'] + x['Açaí-Guaraná'] + x['Pêssego'] ), axis=1)
df_teste = df_teste.drop(['Tempo', 'Estação', 'Movimentação'],  axis=1)
df_teste.head(50)


,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,total_un
0,-23.5489,-46.6388,22,65,22,14,3,26,43,18,213
1,-23.5489,-46.6388,11,56,16,14,43,20,37,15,212
2,-23.5489,-46.6388,86,51,8,11,41,20,35,12,264
3,-23.5489,-46.6388,76,47,65,11,37,14,32,10,292
4,-23.5489,-46.6388,68,42,60,6,35,13,26,5,255
5,-23.5489,-46.6388,57,35,55,43,31,9,22,43,295
6,-23.5489,-46.6388,45,33,50,39,25,5,16,39,252
7,-23.5489,-46.6388,34,28,41,39,22,43,13,34,254
8,-23.5489,-46.6388,22,24,31,35,18,40,8,29,207
9,-23.5489,-46.6388,9,16,26,33,13,38,4,24,163


In [198]:
X_train = df.drop("TARGET", axis=1)
Y_train = df["TARGET"]
X_test = df_teste

In [200]:
#Balanceamento das classes com menos registros no dataset treino
smt = SMOTE()
X_train, Y_train = smt.fit_sample(X_train,Y_train)

In [201]:
#Normalização dos dados
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


### Treinando um classificador com base em no modelo Random Forest

In [212]:
#Algorítomo Machine Learning 
random_forest = RandomForestClassifier(n_estimators=80, max_depth=11, max_features=5, bootstrap=True, min_samples_leaf=3, min_samples_split=7)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)

#importantes = pd.DataFrame(random_forest.feature_importances_, index= X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
#importantes
# Acurácia alcançada pela árvore de decisão
#print("Acurácia: {}%".format(100*round(accuracy_score(y_test, Y_pred), 2)))

In [213]:
# Acurácia alcançada pela árvore de decisão
round(random_forest.score(X_train, Y_train) * 100, 2)

96.04

### Submissão

In [210]:
result = df_teste.assign(TARGET=Y_pred)
result.head(50)

,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,total_un,TARGET
0,-23.5489,-46.6388,22,65,22,14,3,26,43,18,213,REABASTECER
1,-23.5489,-46.6388,11,56,16,14,43,20,37,15,212,REABASTECER
2,-23.5489,-46.6388,86,51,8,11,41,20,35,12,264,REABASTECER
3,-23.5489,-46.6388,76,47,65,11,37,14,32,10,292,NORMAL
4,-23.5489,-46.6388,68,42,60,6,35,13,26,5,255,REABASTECER
5,-23.5489,-46.6388,57,35,55,43,31,9,22,43,295,NORMAL
6,-23.5489,-46.6388,45,33,50,39,25,5,16,39,252,REABASTECER
7,-23.5489,-46.6388,34,28,41,39,22,43,13,34,254,NORMAL
8,-23.5489,-46.6388,22,24,31,35,18,40,8,29,207,NORMAL
9,-23.5489,-46.6388,9,16,26,33,13,38,4,24,163,REABASTECER


In [211]:
#Exportar submissão
result.to_csv('results.csv', index=False)